<center>
    <h1>CMSC320 Final Project</h1>
    Valentyn Bolotin, Jacob Barats, Evan May
</center>
<hr>

<center>
    <h3>Introduction</h3>
</center>
<ol>
    <li>Introduce our topic</li>
    <li>Significance of topic</li>
    <li>Why this topic is important to us</li>
    <li>Explain what we are trying to show with our analysis</li>
</ol>

<hr>

<center>
    <h3>Data collection/curation</h3>
</center>
<ol>
    <li>Explain how we looked for data</li>
    <li>Show/Explain data that we passed up on and give reasons as to why</li>
    <li>Explain why we chose the data source that we used</li>
</ol>

<hr>

<center>
    <h3>Parsing and Data management</h3>
</center>
<ol>
    <li>Why we used Google Collab</li>
    <li>What imports did we use</li>
    <li>Code used to import the data</li>
    <li>Explanations of what the columns mean (the important columns that we will use)</li>
    <li>Did we need to parse the data at all?</li>
    <li>Show how we checked if there were any missing values and how we handled them</li>
    <li>Did we need to tidy the data/remove unnessesary data</li>
  </ol>
<hr>

<center>
    <h3>Exploratory data analysis</h3>
</center>
<ol>
    <li>How did we want to display the data</li>
    <li>Code of displaying data</li>
    <li>Graphs/Charts of displayed data</li>
    </ol>
There will be multiple groups of these three sections
<hr>


<center>
    <h3>Hypothesis testing</h3>
</center>
<ol>
    <li>Conduct hypothesis testing</li>
    <li>Code of displaying data</li>
    <li>Show results in multiple formats (graphs, data, etc)</li>
  </ol>
<hr>

<center>
    <h3>Communication of insights attained</h3>
</center>
  <ol>
    <li>Descirbe what this data means (give multiple interpretations)</li>
    <li>Give reasoning as to why the results were like this (any error that could have influenced results?)</li>
    <li>Conclusion</li>
  </ol>
<hr>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns